** NEEDS azure.ai.projects.models to support ConnectedAgentTool and MessageRole **

https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/connected-agents?pivots=python

# Creating a multi-agent setup
To create a multi-agent setup, follow these steps:

1. Initialize the client object.

In [ ]:
import os
from azure.ai.projects import AIProjectClient
# from azure.ai.projects.models import ConnectedAgentTool, 
from azure.ai.projects.models import MessageRole
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv()


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

2. Create an agent that will be connected to a "main" agent.

In [ ]:
# stock_price_agent = project_client.agents.create_agent(
#     model=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
#     name="stock_price_bot",
#     instructions="Your job is to get the stock price of a company. If you don't know the realtime stock price, return the last known stock price.",
#     #tools=... # tools to help the agent get stock prices
# )

3. Initialize the connected agent tool with the agent ID, name, and description

In [ ]:
# connected_agent = ConnectedAgentTool(
#     id=stock_price_agent.id, name=connected_agent_name, description="Gets the stock price of a company"
# )

4. Create the "main" agent that will use the connected agent.

In [ ]:
# Need azure.ai.projects.models lib upgrade when available to use MessageRole

# agent = project_client.agents.create_agent(
#     model=os.environ["O_MODEL_DEPLOYMENT_NAME"],
#     name="my-agent",
#     instructions="You are a helpful agent, and use the available tools to get stock prices.",
#     tools=connected_agent.definitions,
# )

# print(f"Created agent, ID: {agent.id}")

agent = project_client.agents.get_agent(
    agent_id="asst_vkQdEvbYGAtNZxSz60dNTqPR",
)

5. Create a thread and add a message to it.

In [ ]:
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Need azure.ai.projects.models lib upgrade when available to use MessageRole

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role=MessageRole.USER,
    content="What is the stock price of Microsoft?",
)

print(f"Created message, ID: {message.id}")

6. Create a run and wait for it to complete.

In [ ]:
# Create and process Agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Delete the Agent when done
# project_client.agents.delete_agent(agent.id)
# print("Deleted agent")

# Delete the connected Agent when done
# project_client.agents.delete_agent("stock_price_agent.id")
# print("Deleted connected agent")

7. Print the agent's response. The main agent will compile the responses from the connected agents and provide the response. connected agent responses are only visible to the main agent, and not to the end user.

In [ ]:
# Print the Agent's response message with optional citation
response_message = project_client.agents.list_messages(thread_id=thread.id).get_last_message_by_role(
    MessageRole.AGENT
)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")